<a href="https://colab.research.google.com/github/veotani/ml-univsersity-course-ms-3sem/blob/master/Improved_Matrix_Decomposition_readable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd
import numpy as np

from zipfile import ZipFile
from os import listdir
from scipy.sparse import csr_matrix, find
from scipy.sparse.linalg import svds
from matplotlib import pyplot
from tqdm import tqdm_notebook
from sklearn.neighbors import NearestNeighbors

In [0]:
# Считываем данные
INPUT_FILES_PATH = 'drive/My Drive/study-science/Интеллектуальный анализ данных/'

users =   [] # user ids   translated into row indeces (which are 1 less than id)
movies =  [] # movies ids translated into col indeces (which are 1 less than id)
ratings = [] # rating[k] = (rating that users[k] user gave to movies[k] movie)

new_user_ids = dict()
users_count = 0

for file_part in range(1, 5): # the training data is splited into 4 parts
  filename = INPUT_FILES_PATH + f'combined_data_{file_part}.txt'
  with open(filename) as f:
    for line in f:
      if ':' in line:
        current_movie = int(line.split(':')[0]) - 1
        continue
      user, rating, _ = line.split(',')
      if user in new_user_ids:
        current_user = new_user_ids[user]
      else:
        new_user_ids[user] = users_count
        current_user = users_count
        users_count += 1
      current_rating = int(rating)
      users.append(current_user)
      movies.append(current_movie)
      ratings.append(current_rating)

# Возьмём 10% датасета
max_user_id = ((max(users) + 1)) // 10
max_movie_id = ((max(movies)) + 1) // 10

users_subset = []
movies_subset = []
ratings_subset = []

for i in range(len(users)):
  if users[i] < max_user_id and movies[i] < max_movie_id:
    users_subset.append(users[i])
    movies_subset.append(movies[i])
    ratings_subset.append(ratings[i])

# Удаляем полные данные
del users
del movies
del ratings

# Разделяем на тестовые и тренировочные
test_users = [users_subset[i] for i in range(len(users_subset)) if i % 5 == 1]
train_users = [users_subset[i] for i in range(len(users_subset)) if i % 5 != 1]
del users_subset

test_movies = [movies_subset[i] for i in range(len(movies_subset)) if i % 5 == 1]
train_movies = [movies_subset[i] for i in range(len(movies_subset)) if i % 5 != 1]
del movies_subset

test_ratings = [ratings_subset[i] for i in range(len(ratings_subset)) if i % 5 == 1]
train_ratings = [ratings_subset[i] for i in range(len(ratings_subset)) if i % 5 != 1]
del ratings_subset

ratings_matrix = csr_matrix((train_ratings, (train_movies, train_users)))
del train_ratings
del train_movies
del train_users

# Исправленный метод декомпозиции матрицы

Для поиска результирующих матриц расссмотрим следующую задачу минимизации. 

\\( (X, Y) = argmin_{(X,Y)}\sum_{(i, j) \in E}(X_{i, }\cdot Y_{, j} - A_{i, j})+||X|| + ||Y||\\)

Для её решения воспользуемся методом градиентного спуска с меняющейся длиной шага \\( \gamma \\). Для этого необходимо иметь производную функции, по которой производится минимизация:

\\( F(X, Y) = \sum_{(i, j) \in E}(X_{i, }\cdot Y_{, j} - A_{i, j})^2+||X|| + ||Y||\\)

\\(E\\) - множество индексов элементов исходной матрицы оценок \\( A \\) для которых элементы существуют (пользователь \\( i \\) посставил оценку фильму \\( j \\))

\\( \frac{\partial F}{\partial X_{n, m}} = 2\sum_{(i, j) \in G_n}Y_{m, j}(X_{i,}Y_{,j}-A_{i,j})+2X_{n,m}\\)

\\( G_n = \{(i, j) \in E: i = n\}\\)

Обратим внимание, что: 
1. \\( X_{i,}Y_{,j}-A_{i,j} = M_{i, j}:=(XY-A)_{i,j}\\)
2. \\( \sum_{j}Y_{m,j}M_{i,j}=(YM^T)_{m, i}\\)
3. Для суммирования по \\(E\\) введём матрицу \\(B: \\)
\begin{cases}
   B_{i,j}=1 &A_{i,j}\neq0\\
   B_{i,j}=0&A_{i,j}=0
 \end{cases}
4. Поэлементное умножение матриц \\(A\\) и \\(B\\) занулит все элементы матрицы \\(M\\), индексы которых \\(\notin E\\). В \\(\verb|numpy|\\) поэлементное перемножение выполняется при помощи \\(\verb|P:=np.multiply(A,B)|\\)

Таким образом можно переписать выражение для производной как:

\\( \frac{\partial F}{\partial X_{n, m}} = ((2Y(M\circ P)^T)_{n,m}+2X_{n,m})_{n,m}\\),

т.е.

\\( \frac{\partial F}{\partial X} = 2Y(M\circ P)^T+2X\\)

Для производной по \\(Y\\) аналоичные вычисления приводят к

\\( \frac{\partial F}{\partial Y} = 2X^T(M\circ P)+2Y\\)

Сама функция стоимости в матричном виде будет записана в виде

\\( F = \sum_{i,j} ((M\circ P)\circ (M\circ P)+X\circ X + Y\circ Y)\\)

In [0]:
def cost_fn(X, Y, A):
  cost = 0
  sub = np.multiply(X.dot(Y) - A, (A!=0).astype('b').toarray())
  cost += np.sum(np.multiply(sub, sub))
  cost += np.sum(np.multiply(X, X))
  cost += np.sum(np.multiply(Y, Y))
  return cost

def cost_fn_DX(X, Y, A):
  # A = array(N, M)
  M = X.dot(Y) - A # M = array(N, M)
  M = np.multiply(M, (A!=0).astype('b').toarray())
  P = Y.dot(M.T) # Y   = array(K, M), M.T = array(M, N), P=array(K, N), 
                 # P.T = array(N, K)
  df_dx = 2*(P.T)
  df_dx = df_dx + 2*X
  return df_dx

def cost_fn_DY(X, Y, A):
  M = X.dot(Y) - A
  M = np.multiply(M, (A!=0).astype('b').toarray())
  P = X.T.dot(M) # X.T=array(K, N), M=array(N, M), P=array(K, M)
  df_dy = 2*(P)
  df_dy = df_dy + 2*Y
  return df_dy

def grad_desc_step(X, Y, A, gamma_x=0.001, gamma_y=0.001):
  fn_dx = cost_fn_DX(X, Y, A)
  new_X = X - gamma_x * fn_dx
  del X
  fn_dy = cost_fn_DY(new_X, Y, A)
  new_Y = Y - gamma_y * fn_dy
  return new_X, new_Y

def grad_desc(X, Y, A, eps=1e-3):
  old_df_dx = cost_fn_DX(X, Y, A)
  old_df_dy = cost_fn_DY(X, Y, A)
  new_X, new_Y = grad_desc_step(X, Y, A)
  new_df_dx = cost_fn_DX(new_X, new_Y, A)
  new_df_dy = cost_fn_DY(new_X, new_Y, A)
  gamma_x = np.sum((new_X - X).T.dot(new_df_dx - old_df_dx))/(np.sum(new_df_dx - old_df_dx)**2)
  del new_df_dx
  gamma_y = np.sum((new_Y - Y).T.dot(new_df_dy - old_df_dy))/(np.sum(new_df_dy - old_df_dy)**2)
  del new_df_dy
  del X
  del Y
  it = 1
  print('First iteration is over.')

  while True:
    old_X, old_Y = new_X, new_Y
    del new_X
    del new_Y
    new_X, new_Y = grad_desc_step(old_X, old_Y, A, gamma_x, gamma_y)
    new_df_dx = cost_fn_DX(new_X, new_Y, A)
    new_df_dy = cost_fn_DY(new_X, new_Y, A)
    it += 1
    if np.linalg.norm(new_X - old_X) < eps and np.linalg.norm(new_Y - old_Y) < eps:
      return new_X, new_Y
    else:
      print(f'Delta of costs at iteration {it}')      
      print('X: ', np.linalg.norm(new_X - old_X))
      print('Y: ', np.linalg.norm(new_Y - old_Y))
    gamma_x = np.sum((new_X - old_X).T.dot(new_df_dx - old_df_dx))/(np.sum(new_df_dx - old_df_dx)**2)
    gamma_y = np.sum((new_Y - old_Y).T.dot(new_df_dy - old_df_dy))/(np.sum(new_df_dy - old_df_dy)**2)
    print(gamma_x, gamma_y)
    if gamma_x == 0:
      gamma_x = 1e-20
    if gamma_y == 0:
      gamma_y = 1e-20
    old_df_dx, old_df_dy = new_df_dx, new_df_dy

In [0]:
k = 20

X = np.divide(np.array(ratings_matrix.mean(axis=1)), k)
X = np.array([X for i in range(k)])[:, :, 0].T

Y = np.divide(np.array(ratings_matrix.mean(axis=0)), k)
Y = np.array([Y for i in range(k)])[:, 0, :]

X, Y = grad_desc(X, Y, ratings_matrix)

In [0]:
def grad_desc_x(next_x, Y, A, gamma=1e-3, precision=1, maxit=10):
  it = 0
  next_grad = cost_fn_DX(next_x, Y, A)
  while True:
    current_x = next_x
    current_grad = next_grad
    next_x = current_x - gamma * current_grad
    next_grad = cost_fn_DX(next_x, Y, A)
    step = np.linalg.norm(next_x - current_x)
    gamma = np.sum((next_x - current_x).T.dot(next_grad - current_grad))/(np.sum(next_grad - current_grad)**2)
    print(step)
    it += 1
    if step <= precision or it >= maxit:
        return current_x

def grad_desc_y(X, next_y, A, gamma=1e-3, precision=1, maxit=10):
  it=0
  next_grad = cost_fn_DY(X, next_y, A)
  while True:
    current_y = next_y
    current_grad = next_grad
    next_y = current_y - gamma * current_grad
    next_grad = cost_fn_DY(X, next_y, A)
    step = np.linalg.norm(next_y - current_y)
    print(step)
    gamma = np.sum((next_y - current_y).T.dot(next_grad - current_grad))/(np.sum(next_grad - current_grad)**2)
    it+=1
    if step <= precision or it>=maxit:
        return current_y

k = 3

X = np.divide(np.array(ratings_matrix.mean(axis=1)), k)
X = np.array([X for i in range(k)])[:, :, 0].T

Y = np.divide(np.array(ratings_matrix.mean(axis=0)), k)
Y = np.array([Y for i in range(k)])[:, 0, :]

new_x = grad_desc_x(X, Y, ratings_matrix)
new_y = grad_desc_y(new_x, Y, ratings_matrix)

In [0]:
new_x = grad_desc_x(new_x, new_y, ratings_matrix)
new_y = grad_desc_y(new_x, new_y, ratings_matrix)

In [0]:
error = 0
predicted_ratings = X*Y
for i in range(len(test_ratings)):
  predicted_rating_estim = predicted_ratings[test_movies[i], test_users[i]]
  if predicted_rating_estim < 1:
    predicted_rating = 1
  elif predicted_rating_estim > 5:
    predicted_rating = 5
  else:
    predicted_rating = int(round(predicted_rating_estim, 0))
  error += abs(test_ratings[i] - predicted_rating)
print(error/len(test_users))

k = 3 => error = 1.56

k = 10 => error = 1.05

k = 7 => error = 1.05

k = 50 => error = 1.59